In [1]:
### Libraries ###

from bs4 import BeautifulSoup
import requests

In [4]:
### Get webpage html text ###

response = requests.get("https://www.dandwiki.com/wiki/Aaeaea_(5e_Creature)")

if response.status_code == 200:                                         # Checks that the connection to the webpage was established
    Aaeaea_soup = BeautifulSoup(response.content, "html.parser")        # Makes the HTMLsoup from the content of the response and assigns it to variable MonsterName_soup
    print(Aaeaea_soup)                                                  # Prints content of the connected webpage
else:
    print(response)                                                     # If not a good connection, prints response code

b'<!DOCTYPE html>\n<html class="client-nojs" lang="en" dir="ltr">\n<head>\n<meta charset="UTF-8"/>\n<title>Aaeaea (5e Creature) - D&amp;D Wiki</title>\n<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":false,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"72d2b54c9e5b0640812ef31b","wgCSPNonce":false,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"Aaeaea_(5e_Creature)","wgTitle":"Aaeaea (5e Creature)","wgCurRevisionId":1651606,"wgRevisionId":1651606,"wgArticleId":218144,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["CR1","5e","Creature","Undead Type","Tiny Size","Swamp Monster","User"],"wgPageContentLanguage":"en","wgPageContentModel":"wikitext","wgReleva